### imports

In [ ]:
from msilib.schema import RadioButton
from time import sleep
from matplotlib import pyplot as plt
import numpy as np
import sklearn
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
from torchvision.datasets import mnist
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch
import keyboard
from matplotlib.widgets import Slider, Button, RadioButtons

import warnings
warnings.filterwarnings("ignore")

# get the data
train_dataset = mnist.MNIST(root='./ass3/train', train=True, transform=ToTensor(), download=True)

# put data into dataloader:
train_loader = DataLoader(train_dataset, batch_size=256)

# make a custom dataloader
class SelfSupervisionDataLoader(DataLoader):
    
    def __init__(self, loader : DataLoader, transform=None, num_classes=10):
        """If transform is not specified, the data is returned as is (for supervised learning).
Otherwise, the data is transformed, and the labels are the original data."""
        self.loader = loader
        self.transform = transform
        self.num_classes = num_classes
    
    def __iter__(self):
        for batch in self.loader:
            data, labels = batch
            # labels = torch.eye(self.num_classes)[labels]
            if self.transform is not None:
                labels = data.clone().double()
                data = self.transform(data)
            yield data, labels

# function to add noise to the data
def addNoise(data, noise_level = 0.3):
    noise = np.random.normal(0, noise_level, size=data.shape)
    # clamp values of data to be between 0 and 1
    data = np.clip(data + noise, 0, 1)
    return data
